In [ ]:
!pip install tensorly

     |████████████████████████████████| 71kB 2.0MB/s 
     |████████████████████████████████| 163kB 7.0MB/s 
  Created wheel for tensorly: filename=tensorly-0.4.5-cp36-none-any.whl size=100156 sha256=12b0f798180dbc63c772d64727db10a36a7ad68f83d6fba4ce62c6e15f90f466
  Stored in directory: /root/.cache/pip/wheels/44/ae/02/8d00229a4fd0af192b48d24da903f9975c7ac10e706685fc39
Successfully built tensorly


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from tqdm.notebook import tqdm

import tensorly as tl
from tensorly.random import random_tucker
from tensorly.tucker_tensor import tucker_to_tensor

tl.set_backend('pytorch')

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class NeuralTensorLayer(torch.nn.Module):
    
    """
    This is the class for 
    """
    
    def __init__(self, order, input_dim, output_dim, rank_tucker=-1,
                 initializer=torch.nn.init.xavier_uniform):
        
        super(NeuralTensorLayer, self).__init__()
        
        self.order = order
        self.rank_tucker = rank_tucker
        
        if order > 3 or order < 1:
            raise Exception('Order must be in range [1, 3]')
            
        if rank_tucker != -1 and rank_tucker < 1:
            raise Exception('Tucker rank must be -1 or greater than 0 integer')
            
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        self.bias = nn.Parameter(torch.zeros((1, output_dim)), requires_grad=True)
        initializer(self.bias)
        
        self.myparameters = torch.nn.ParameterList([self.bias])
        
        self.order1_tens = self.initialize_n_order_tensor(1, initializer)
        
        if order >= 2:
            self.order2_tens = self.initialize_n_order_tensor(2, initializer)
            
        if order == 3:
            self.order3_tens = self.initialize_n_order_tensor(3, initializer)
        
    # initialize tensor in full or in decomposed form and register it as parameter
    def initialize_n_order_tensor(self, order, initializer):
        
        if self.rank_tucker >= 1:
            
            dim_list = [self.input_dim] * order + [self.output_dim]
            tens_core, factors = random_tucker(dim_list, self.rank_tucker)
            tens_core = nn.Parameter(tens_core, requires_grad=True)
            factors = [nn.Parameter(fact, requires_grad=True) for fact in factors]
            
            self.myparameters.append(tens_core)
            for fact in factors:
                self.myparameters.append(fact)
                
            return (tens_core, factors)
            
        else:
            
            dim_list = [self.input_dim] * order + [self.output_dim]
            var = nn.Parameter(torch.zeros(dim_list), requires_grad=True)
            initializer(var)
            self.myparameters.append(var)
            
            return var

    def compute_result_for_vec(self, core, factor_inp, last_factor): # result dim (1, 1)
        result = core
        for i in range(len(factor_inp)):
            result = tl.tenalg.mode_dot(result, factor_inp[i], i)
        result = result.view(1, -1).mm(torch.transpose(last_factor, 0, 1))
        return result.view(-1)

    def mode_n_dot_accelerated(self, core, factors, input):

        new_factors = [torch.transpose(factors[i], 0, 1).mm(input) for i in range(len(factors) - 1)]

        return torch.stack([
                            self.compute_result_for_vec(core, 
                                                        [new_factors[k][:, i] for k in range(len(factors) - 1)], factors[-1]) 
                            for i in range(input.shape[1])
                            ], dim=0)
        
    def forward(self, X, transposed=False):
        
        X = torch.Tensor(X)
        
        if self.rank_tucker == -1:
            result = torch.addmm(self.bias, X, self.order1_tens)
        else:
            result = torch.addmm(self.bias, X, tucker_to_tensor(self.order1_tens))
        
        if self.order >= 2:
            
            if self.rank_tucker == -1:      
                acc = tl.tenalg.mode_dot(self.order2_tens, X, 0)
            else:
                acc = tl.tenalg.mode_dot(tucker_to_tensor(self.order2_tens), X, 0)

            acc = tl.tenalg.mode_dot(acc, X, 1)
            result += torch.einsum('iik->ik', acc)
        
        if self.order == 3:
             
            if self.rank_tucker == -1:      
                acc = tl.tenalg.mode_dot(self.order3_tens, X, 0)
            else:
                acc = tl.tenalg.mode_dot(tucker_to_tensor(self.order3_tens), X, 0)
            
            acc = tl.tenalg.mode_dot(acc, X, 1)
            acc = tl.tenalg.mode_dot(acc, X, 2)
            result += torch.einsum('iiik->ik', acc)
        
        res = result.reshape((X.shape[0], self.output_dim))
        return res

In [ ]:
def func(a, b, c, data):
    return a * data[:, 0] + b * data[:, 1] + c

def generate_linear_data(N, a, b, c, noise):
    inp = np.random.randn(N, 2)
    vals = func(a, b, c, inp) + np.random.randn(N) * noise
    return inp, vals

data = generate_linear_data(2000, 0.3, 4, 40, 10)

In [ ]:
import pandas as pd

In [ ]:
dir = '/content/drive/My Drive/study/Skoltech/MTF/Project/'

In [ ]:
data = pd.read_csv(dir + 'diabetes.csv')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
target = np.matrix(data['Outcome']).T
data = np.matrix(data.drop('Outcome', axis=1))

In [ ]:
input_dim = data.shape[1]
output_dim = 1

In [ ]:
input_dim

8

In [ ]:
NN = torch.nn.Sequential(NeuralTensorLayer(2, input_dim, 2, rank_tucker=5), 
                         nn.BatchNorm1d(2),
                         nn.Softmax(dim=-1))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


In [ ]:
loss = torch.nn.CrossEntropyLoss()

In [ ]:
NN

Sequential(
  (0): NeuralTensorLayer(
    (myparameters): ParameterList(
        (0): Parameter containing: [torch.FloatTensor of size 1x2]
        (1): Parameter containing: [torch.FloatTensor of size 5x5]
        (2): Parameter containing: [torch.FloatTensor of size 8x5]
        (3): Parameter containing: [torch.FloatTensor of size 2x5]
        (4): Parameter containing: [torch.FloatTensor of size 5x5x5]
        (5): Parameter containing: [torch.FloatTensor of size 8x5]
        (6): Parameter containing: [torch.FloatTensor of size 8x5]
        (7): Parameter containing: [torch.FloatTensor of size 2x5]
    )
  )
  (1): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Softmax(dim=-1)
)

In [ ]:
optimizer = torch.optim.Adam(NN.parameters(), lr=0.001)

In [ ]:
torch.cuda.is_available()

True

In [ ]:
num_iter = 1000
batch_size = 100
errors = []
batch_count = len(data) // batch_size if len(data) % batch_size == 0 else len(data) // batch_size + 1

for i in tqdm(range(num_iter)):
    
    mean_loss = 0
    
    for batch in range(batch_count):
        
        inp = torch.Tensor(data[batch * batch_size:(batch + 1) * batch_size])
        tar = torch.Tensor(target[batch * batch_size:(batch + 1) * batch_size]).type(torch.long)
    
        optimizer.zero_grad()
        output = NN(inp)
        loss_val = loss(output, torch.flatten(tar))
        loss_val.backward()
        optimizer.step()
        
        mean_loss += loss_val
        
    errors.append(mean_loss / batch_count)
        
    if i % 100 == 0:
        print(loss_val)

tensor(0.6921, grad_fn=<NllLossBackward>)
tensor(0.5253, grad_fn=<NllLossBackward>)
tensor(0.5261, grad_fn=<NllLossBackward>)
tensor(0.5254, grad_fn=<NllLossBackward>)
tensor(0.5241, grad_fn=<NllLossBackward>)
tensor(0.5209, grad_fn=<NllLossBackward>)
tensor(0.5150, grad_fn=<NllLossBackward>)
tensor(0.5056, grad_fn=<NllLossBackward>)
tensor(0.4804, grad_fn=<NllLossBackward>)
tensor(0.4694, grad_fn=<NllLossBackward>)



In [ ]:
A = torch.randn(3, 3, 4)
torch.einsum('iik->ik', A)

In [ ]:
A[1, 1]

In [ ]:
plt.figure(figsize=(15, 15))
plt.plot(np.arange(len(errors)), errors)

In [ ]:
inp = torch.Tensor(data)
tar = torch.Tensor(target).type(torch.long).reshape(target.shape[0])

In [ ]:
output = NN(inp)

In [ ]:
(output.argmax(axis=1) == tar).sum().item() / len(tar)

0.7877604166666666

# Now, when the NN is ready, we can think about low rank approximation approaches

In [ ]:
random_tucker([8, 1], 5)

(tensor([[0.5533, 0.2749, 0.2266, 0.3726, 0.6167],
         [0.1750, 0.2642, 0.8940, 0.2651, 0.1726],
         [0.3079, 0.4410, 0.6247, 0.3316, 0.3537],
         [0.0400, 0.1314, 0.5639, 0.3560, 0.8727],
         [0.6288, 0.1125, 0.6745, 0.5497, 0.3638]]),
 [tensor([[0.2792, 0.2677, 0.4475, 0.5355, 0.4844],
          [0.4759, 0.7013, 0.8256, 0.5355, 0.3042],
          [0.4108, 0.0907, 0.1525, 0.9658, 0.4803],
          [0.4368, 0.6058, 0.6352, 0.3549, 0.5398],
          [0.9099, 0.6247, 0.7035, 0.8182, 0.9224],
          [0.2291, 0.8454, 0.9994, 0.4533, 0.6601],
          [0.8346, 0.6404, 0.1583, 0.1318, 0.9663],
          [0.9368, 0.7839, 0.4672, 0.3892, 0.7628]]),
  tensor([[0.7744, 0.1758, 0.2186, 0.1950, 0.3399]])])